In [2]:
# https://www.codingame.com/ide/puzzle/binary-tree-visual


class BinaryNode:
    def __init__(A, x=None, left=None, right=None, parent=None):
        A.left = left
        A.right = right
        A.parent = parent
        A.val = x
        
        A.link_parents()
    
    def __repr__(self): return f'BinaryNode({self.val}, left={self.left}, right={self.right})'
    
    def link_parents(A):
        if A.left and not A.left.parent:
            A.left.parent = A
            A.left.link_parents()
        
        if A.right and not A.right.parent:
            A.right.parent = A
            A.right.link_parents()
        return A

    def l(A): return list(A.subtree_iter())

    def subtree_iter(A): # O(n)
        if A.left: yield from A.left.subtree_iter()
        yield A.val
        if A.right: yield from A.right.subtree_iter()


from queue import deque


def old_list2tree(root_list):
    root = BinaryNode(root_list[0])

    node = root
    queue = deque()
    state = 0

    for val in root_list[1:]:
        # print(queue)
        if state == 2:
            node = queue.pop()
            state = 0

        if val is None:
            state += 1
            continue

        new_node = BinaryNode(val)


        if state == 0:
            node.left = new_node
        else:
            node.right = new_node
        state += 1

        queue.appendleft(new_node)


    return root

null = None
def list2tree(li):
    if not li: return None

    the_root = BinaryNode(li[0])
    r = None

    q = deque([the_root])
    left_flag = False
    
    for i in range(1, len(li)):
        left_flag = not left_flag
        if left_flag:
            r = q.popleft()

        if li[i] is None: continue
        child = BinaryNode(li[i])
        q.append(child)

        if left_flag:
            r.left = child
        else:
            r.right = child

    the_root.link_parents()
    return the_root


li = [2,null,3,4,null,5]
# print(list2tree(li))
# print(old_list2tree(li))
# tree = BinaryNode('111', BinaryNode('222', BinaryNode('44'), BinaryNode('555', None, BinaryNode('66'))), BinaryNode('3'))
tree = list2tree([111, 222, 3, 44, 555,null,null,null,66])
# print(tree)


class BinaryTreeASCIIArt:
    def __init__(self, rows, root_i=None):
        self.rows = rows
        self.root_i = root_i
    
    def __repr__(self):
        return '\n'.join(s.rstrip() for s in self.rows)

print(
BinaryTreeASCIIArt([
    '   222',
    '     |',
    '  +--+',
    '  |',
    ' 44'
])
)



   222
     |
  +--+
  |
 44


In [3]:
def pad_right(s, upto, symbol=' '):
    pad = max(0, upto-len(s))
    return s+symbol*pad


def pad_left(s, upto, symbol=' '):
    pad = max(0, upto-len(s))
    return symbol*pad+s

pad_left('hey', 4, '-'), pad_right('hey', 4, '-')

('-hey', 'hey-')

In [187]:
def grow_connections(root_row, left, right):
    connections = []
    
    if not left and not right: return connections

    # FIRST
    connections.append( pad_left('|', len(root_row)) )


    # SECOND
    elbow = '+'
    
    # SECOND: LEFT
    if left:
        elbow_len = len(root_row) - left.root_i
        elbow = '+'+('-'*(elbow_len-2)) + elbow

    elbow = pad_left(elbow, len(root_row))
    
    # SECOND: RIGHT
    if right:
        elbow_len = right.root_i + 2
        elbow += ('-'*(elbow_len-2)) + '+'

    connections.append( elbow )

    # THIRD
    bottom = ''
    if left:
        bottom += pad_left('|', left.root_i+1)
        
    bottom = pad_right(bottom, len(root_row))
    if right:
        bottom += pad_left('|', right.root_i+1)
        
    # bottom = pad_left
    connections.append( bottom )
    return connections

In [194]:
import itertools

def merge_rows(left_rows, right_rows, left_pad_n):
    res = []
    for l,r in itertools.zip_longest(left_rows, right_rows, fillvalue=''):
        l = pad_right(l, left_pad_n)
        res.append(l + r)
    return res


def _tree2ascii(root: BinaryNode, W) -> BinaryTreeASCIIArt:
    if not root: return None

    rows = []
    root_pad_n = W

    left = _tree2ascii(root.left, W)
    right = _tree2ascii(root.right, W)

    if left:
        root_pad_n += max(map(len, left.rows))
        rows += left.rows


    root_row = pad_left(str(root.val), root_pad_n)


    if right:
        rows = merge_rows(rows, right.rows, root_pad_n)


    connections = grow_connections(root_row, left, right)


    our_rows = [root_row] + connections + rows
    return BinaryTreeASCIIArt(our_rows, len(root_row)-1)


def max_val_len(root):
    if not root: return 0

    return max(
        len(str(root.val)),
        max_val_len(root.left),
        max_val_len(root.right),
    )

def tree2ascii(root):
    W = 1+max_val_len(root)
    return _tree2ascii(root, W)


In [195]:
root = BinaryNode(333, 
                  left=BinaryNode(333),
                right=BinaryNode(44, BinaryNode(33))
        )

In [196]:
tree2ascii(root)

     333
       |
   +---+-------+
   |           |
 333          44
               |
           +---+
           |
          33

In [198]:
tree2ascii(list2tree([111, 222, 3, 33, 555, None, None, None, None, None, 66]))

                 111
                   |
       +-----------+---+
       |               |
     222               3
       |
   +---+---+
   |       |
  33     555
           |
           +---+
               |
              66

     333
       |
   +---+-------+
   |           |
 333          44
               |
           +---+
           |
          33

In [163]:
len(right.rows[0])

4